In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import tqdm 
import matplotlib.pyplot as plt
import math
import scipy
import re

In [2]:
import xgboost as xgb
import lightgbm as lgb
import catboost as cat

In [3]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder,OrdinalEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [4]:
from sklearn.metrics import roc_auc_score, roc_curve,confusion_matrix

In [5]:
pd.set_option('display.max_columns', None)

# Loading Raw Data

In [6]:
raw_data = pd.read_csv('telecomChurn.zip')

In [7]:
raw_data.head(5)

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,opk_dat_Mean,mou_opkv_Mean,mou_opkd_Mean,drop_blk_Mean,attempt_Mean,complete_Mean,callfwdv_Mean,callwait_Mean,churn,months,uniqsubs,actvsubs,new_cell,crclscod,asl_flag,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,prizm_social_one,area,dualband,refurb_new,hnd_price,phones,models,hnd_webcap,truck,rv,ownrent,lor,dwlltype,marital,adults,infobase,income,numbcars,HHstatin,dwllsize,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID
0,23.9975,219.25,22.500,0.2475,0.00,0.0,0.0,0.0,0.0,-157.25,-18.9975,0.666667,0.0,0.666667,0.0,6.333333,0.0,52.333333,0.0,42.333333,0.0,45.000000,0.0,0.000000,0.000000,0.000000,18.000000,0.000000,90.643333,0.0,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,0.0,132.600000,0.0,24.000000,0.0,55.220000,0.0,1.333333,52.333333,45.000000,0.0,0.333333,1,61,2,1,U,A,N,1652,4228.00000,1504.62,1453.44,4085.00,1602,29.66,83.37,32.69,272,116,30,322.0,136.0,38.0,S,NORTHWEST/ROCKY MOUNTAIN AREA,Y,N,149.98999,2.0,2.0,WCMB,0.0,0.0,O,15.0,S,S,1.0,M,4.0,3.0,C,A,0.0,N,U,U,U,U,U,Y,361.0,1000001
1,57.4925,482.75,37.425,0.2475,22.75,9.1,9.1,0.0,0.0,532.25,50.9875,8.333333,0.0,1.000000,0.0,61.333333,0.0,263.333333,0.0,69.000000,0.0,193.333333,0.0,1.666667,6.333333,5.463333,53.000000,0.333333,189.396667,0.0,55.280000,46.333333,24.216667,6.333333,3.696667,83.666667,0.0,75.333333,0.0,157.000000,0.0,169.343333,0.0,9.333333,263.333333,193.333333,0.0,5.666667,0,56,1,1,N,EA,N,14654,26400.00000,2851.68,2833.88,26367.00,14624,51.53,479.40,265.89,305,158,40,477.0,275.0,48.0,U,CHICAGO AREA,N,N,NaN,7.0,6.0,WC,1.0,1.0,NaN,1.0,S,S,1.0,M,5.0,1.0,C,A,0.0,Z,U,U,U,U,U,Y,240.0,1000002
2,16.9900,10.25,16.990,0.0000,0.00,0.0,0.0,0.0,0.0,-4.25,0.0000,0.333333,0.0,0.000000,0.0,2.666667,0.0,9.000000,0.0,0.333333,0.0,6.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,5.426667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.0,5.193333,0.0,1.000000,0.0,0.233333,0.0,0.333333,9.000000,6.000000,0.0,0.000000,1,58,1,1,Y,C,N,7903,24385.05333,2155.91,1934.47,24303.05,7888,34.54,433.98,140.86,12,7,17,11.0,6.0,17.0,S,GREAT LAKES AREA,N,N,29.98999,2.0,1.0,NaN,0.0,0.0,O,7.0,S,M,2.0,M,5.0,2.0,C,A,0.0,N,U,Y,U,U,U,Y,1504.0,1000003
3,38.0000,7.50,38.000,0.0000,0.00,0.0,0.0,0.0,0.0,-1.50,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.0,3.666667,0.0,1.333333,0.0,3.666667,0.0,0.000000,0.000000,0.000000,1.333333,0.000000,8.410000,0.0,0.413333,0.333333,0.256667,0.000000,0.000000,1.333333,0.0,3.380000,0.0,3.666667,0.0,5.450000,0.0,0.000000,3.666667,3.666667,0.0,0.000000,0,60,1,1,Y,B,N,1502,3065.00000,2000.90,1941.81,3035.00,1479,40.45,63.23,30.81,8,3,38,50.0,25.0,40.0,T,CHICAGO AREA,N,N,29.98999,1.0,1.0,NaN,0.0,0.0,NaN,6.0,M,M,4.0,M,6.0,1.0,C,D,0.0,U,Y,U,U,U,U,Y,1812.0,1000004
4,55.2300,570.50,71.980,0.0000,0.00,0.0,0.0,0.0,0.0,38.50,0.0000,9.666667,0.0,0.666667,0.0,77.000000,0.0,222.333333,0.0,94.666667,0.0,137.000000,0.0,8.666667,15.000000,11.076667,66.000000,0.000000,285.233333,0.0,106.330000,14.666667,10.816667,0.666667,0.366667,97.333333,0.0,173.476667,0.0,90.333333,0.0,218.086667,0.0,10.333333,222.333333,137.000000,0.0,0.000000,0,57,1,1,Y,A,N,4485,14028.00000,2181.12,2166.48,13965.00,4452,38.69,249.38,79.50,558,191,55,586.0,196.0,80.0,U,NEW ENGLAND AREA,Y,N,149.98999,6.0,4.0,WCMB,0.0,0.0,R,5.0,M,S,1.0,M,6.0,1.0,C,O,0.0,I,U,U,U,U,U,Y,434.0,1000005


## Handling column descriptions

Since there is literally a hundred columns in this dataset, lets take a general look at what kind of data is available. 

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import OPTICS, KMeans
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import spacy

### read-in column descriptions

In [43]:
col_desc = pd.read_json('columnDescriptions.json',orient='index')[0].rename('col_desc').to_frame()

In [44]:
col_desc.head(5)

,col_desc
rev_Mean,Mean monthly revenue (charge amount)
mou_Mean,Mean number of monthly minutes of use
totmrc_Mean,Mean total monthly recurring charge
da_Mean,Mean number of directory assisted calls
ovrmou_Mean,Mean overage minutes of use


### Preprocess column descriptions

In [45]:
col_desc['clean_desc'] = (
    col_desc['col_desc']
    .apply(gensim.parsing.preprocessing.remove_stopwords)
)

In [46]:
nlp = spacy.load('en_core_web_sm')

In [47]:
col_desc['clean_desc'].apply(lambda x: [token.lemma_ for token in nlp(x)])

rev_Mean       [mean, monthly, revenue, (, charge, amount, )]
mou_Mean                 [mean, number, monthly, minute, use]
totmrc_Mean             [mean, total, monthly, recur, charge]
da_Mean               [mean, number, directory, assist, call]
ovrmou_Mean                      [mean, overage, minute, use]
                                    ...                      
kid11_15             [child, 11, -, 15, year, age, household]
kid16_17             [child, 16, -, 17, year, age, household]
creditcd                            [credit, card, indicator]
eqpdays          [number, day, (, age, ), current, equipment]
Customer_ID                                               [n]
Name: clean_desc, Length: 100, dtype: object

### Bag of words

In [12]:
tokenizedDesc = col_desc['clean_desc'].apply(lambda x: x.split()).to_list()
dictionary = gensim.corpora.Dictionary(tokenizedDesc)
corpus = [dictionary.doc2bow(text) for text in tokenizedDesc]

### Tfidf

In [13]:
tfidf = TfidfVectorizer()
tfidfMatrix = tfidf.fit_transform(col_desc['clean_desc'])

In [14]:
tfidf.get_feature_names_out()

array(['10', '11', '15', '16', '17', '31', '60', 'account', 'active',
       'adjusted', 'adults', 'age', 'amount', 'area', 'assisted',
       'attempted', 'average', 'billing', 'blocked', 'calls',
       'capability', 'card', 'care', 'cell', 'change', 'charge', 'child',
       'churn', 'class', 'code', 'completed', 'credit', 'current',
       'custcare_mean', 'customer', 'data', 'date', 'days', 'directory',
       'dropped', 'dualband', 'dummy', 'dwelling', 'equipment',
       'estimated', 'ethnicity', 'failed', 'foreign', 'forwarding',
       'geogrpahic', 'group', 'handset', 'handsets', 'home', 'household',
       'inbound', 'income', 'indicator', 'infobase', 'instance', 'issued',
       'known', 'length', 'letter', 'life', 'limit', 'marital', 'match',
       'mean', 'minute', 'minutes', 'models', 'month', 'monthly',
       'months', 'new', 'number', 'observation', 'off', 'outbound',
       'overage', 'owner', 'peak', 'percentage', 'phone', 'placed',
       'premier', 'previous', 'p

### Optics grouping

In [45]:
optics = OPTICS(min_samples=4)
col_desc['optics_class'] = optics.fit_predict(tfidfMatrix.toarray())

In [46]:
col_desc.optics_class.value_counts()

-1    62
 3    14
 0     8
 1     6
 4     5
 2     5
Name: optics_class, dtype: int64

In [47]:
for groupId in np.unique(optics.labels_):
    gensim.models.ldamodel.LdaModel(
        (pd.Series(corpus)[col_desc['optics_class'].reset_index(drop=True)==0]).to_list(),
        num_topics = 3, id2word=dictionary, passes=12
    )

In [56]:
lda = gensim.models.ldamodel.LdaModel(
    (pd.Series(corpus)[col_desc['optics_class'].reset_index(drop=True)==0]).to_list(),
    num_topics = 1, id2word=dictionary, passes=12)

In [54]:
re.findall("(\".+?\")",lda.print_topics()[0][1])

['"life"',
 '"customer"',
 '"Total"',
 '"calls"',
 '"minutes"',
 '"use"',
 '"number"',
 '"Average"',
 '"revenue"',
 '"monthly"']

In [20]:
col_desc[col_desc.optics_class==0]

,col_descriptions,clean_desc,optics_class
totcalls,Total number of calls over the life of the cus...,Total number calls life customer,0
totmou,Total minutes of use over the life of the cust...,Total minutes use life customer,0
adjrev,Billing adjusted total revenue over the life o...,Billing adjusted total revenue life customer,0
adjmou,Billing adjusted total minutes of use over the...,Billing adjusted total minutes use life customer,0
adjqty,Billing adjusted total number of calls over th...,Billing adjusted total number calls life customer,0
avgrev,Average monthly revenue over the life of the c...,Average monthly revenue life customer,0
avgmou,Average monthly minutes of use over the life o...,Average monthly minutes use life customer,0
avgqty,Average monthly number of calls over the life ...,Average monthly number calls life customer,0


### kmeans grouping

In [172]:
kmeans = KMeans(n_clusters=10)
col_desc['kmeans_class'] = kmeans.fit_predict(X.toarray())

In [173]:
col_desc['kmeans_class'].value_counts()

4    48
0    11
1    10
3     8
9     6
8     5
5     5
6     3
7     2
2     2
Name: kmeans_class, dtype: int64

In [174]:
ls = []
for group in col_desc['kmeans_class'].unique():
    ls.append(col_desc[col_desc['kmeans_class']==group]['col_desc'].to_numpy())

In [211]:
col_desc[col_desc['kmeans_class']==0]['col_desc'].to_numpy()

array(['Mean number of monthly minutes of use',
       'Mean overage minutes of use',
       'Mean rounded minutes of use of customer care calls',
       'Mean unrounded minutes of use of customer care (see CUSTCARE_MEAN) calls',
       'Mean unrounded minutes of use of completed voice calls',
       'Mean unrounded minutes of use of completed data calls',
       'Mean unrounded minutes of use of received voice calls ',
       'Mean unrounded minutes of use of peak voice calls',
       'Mean unrounded minutes of use of peak data calls',
       'Mean unrounded minutes of use of off-peak voice calls',
       'Mean unrounded minutes of use of off-peak data calls'],
      dtype=object)